# Salary Predictions Based on Job Descriptions

# Part 1 - DEFINE

### ---- 1 Define the problem ----

### CORRELATING SKILL WITH SALARY 

The prediction of Salary structure in Human Resource department ( HR) of a company  contains two fold objectives: 

    -To  attract and retain the talented employees 
    
The concept of demand and supply of work force  needs  to be examined. The key intakes in HR department include - identifying the responsibilities of each  job, appointing the right man to the right post  and evolving a salary package that suits the best in the industry,  based on parameters like qualifications, industry, job location.  This is vital to the HR department as it helps the company in reducing the unnecessary cost while retaining the best talent. 

On the other hand, Salary prediction can also help the job seekers. It helps job seekers to  take more informed decisions and update their skills/knowledge to maximize salary.

**It is a win – win formula for both employers and job seekers.** 

In [9]:
#import libraries

#Analysis packages
import pandas as pd
import sklearn as sk
import numpy as np

#Visualization packages
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

#Sklearn packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor


#your info here
__author__ = "Rahul Vashsith"
__email__ = "rahulvashisth3@gmail.com"

## Part 2 - DISCOVER

### ---- 2 Load the data ----

In [10]:
#loading the data into Pandas dataframe

#Training Dataframes
train_features = pd.read_csv(r"/mnt/c/Users/rahul/Desktop/DSDJ/Module_4/4.5/Instructions_and_Notebook/salarypredictionportfolio/train_features.csv")
train_salaries = pd.read_csv(r"/mnt/c/Users/rahul/Desktop/DSDJ/Module_4/4.5/Instructions_and_Notebook/salarypredictionportfolio/train_salaries.csv")

#Testing Dataframes
test_features = pd.read_csv(r"/mnt/c/Users/rahul/Desktop/DSDJ/Module_4/4.5/Instructions_and_Notebook/salarypredictionportfolio/test_features.csv")


In [11]:
#merging train_features and train_salary
df = train_features.merge(train_salaries, on='jobId', how='left')

In [13]:
df.head(5)

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17,142
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16,163


In [7]:
#test dataset
test_features.head(5)

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362685407687,COMP33,MANAGER,HIGH_SCHOOL,NONE,HEALTH,22,73
1,JOB1362685407688,COMP13,JUNIOR,NONE,NONE,AUTO,20,47
2,JOB1362685407689,COMP10,CTO,MASTERS,BIOLOGY,HEALTH,17,9
3,JOB1362685407690,COMP21,MANAGER,HIGH_SCHOOL,NONE,OIL,14,96
4,JOB1362685407691,COMP36,JUNIOR,DOCTORAL,BIOLOGY,OIL,10,44


### ---- 3 Clean the data ----

In [17]:
#summary statistics for each varible 
df.describe(include='all')

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
count,1000000,1000000,1000000,1000000,1000000,1000000,1000000.000000,1000000.000000,1000000.000000
unique,1000000,63,8,5,9,7,NaN,NaN,NaN
top,JOB1362685301887,COMP39,SENIOR,HIGH_SCHOOL,NONE,WEB,NaN,NaN,NaN
freq,1,16193,125886,236976,532355,143206,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,11.992386,49.529260,116.061818
std,NaN,NaN,NaN,NaN,NaN,NaN,7.212391,28.877733,38.717936
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,25.000000,88.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,50.000000,114.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,18.000000,75.000000,141.000000


In [18]:
#check dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
 8   salary               1000000 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 76.3+ MB


In [19]:
test_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [20]:
#checking for duplicate data and removing it
df[df.duplicated()]

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary


In [21]:
#checking if we have any invalid data- salary < 0 
(df['salary'] <= 0).sum()

5

We have 5 rows where salary < 0, as we are predicting salary these rows are invalid.

In [24]:
#removing salary < 0 
df.drop(df[df['salary'] <=0].index, inplace=True)

In [25]:
#checking if we removed salary <0
df.shape

(999995, 9)


It checks the unique values in each column and counts the repetitions. We can check for missing or incorrect values.

The data is perfect, no missing or incorrect values

In [27]:
#checking for null values 
for i in df:
    print(i)
    print( pd.isnull(df[i]).value_counts())

jobId
False    999995
Name: jobId, dtype: int64
companyId
False    999995
Name: companyId, dtype: int64
jobType
False    999995
Name: jobType, dtype: int64
degree
False    999995
Name: degree, dtype: int64
major
False    999995
Name: major, dtype: int64
industry
False    999995
Name: industry, dtype: int64
yearsExperience
False    999995
Name: yearsExperience, dtype: int64
milesFromMetropolis
False    999995
Name: milesFromMetropolis, dtype: int64
salary
False    999995
Name: salary, dtype: int64


In [28]:
#checking for data types
df.dtypes

jobId                  object
companyId              object
jobType                object
degree                 object
major                  object
industry               object
yearsExperience         int64
milesFromMetropolis     int64
salary                  int64
dtype: object

### ---- 4 Explore the data (EDA) ----

In [3]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

### ---- 5 Establish a baseline ----

In [5]:
#select a reasonable metric (MSE in this case)
#create an extremely simple model and measure its efficacy
#e.g. use "average salary" for each industry as your model and then measure MSE
#during 5-fold cross-validation

### ---- 6 Hypothesize solution ----

In [ ]:
#brainstorm 3 models that you think may improve results over the baseline model based
#on your 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### ---- 7 Engineer features  ----

In [ ]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

### ---- 8 Create models ----

In [15]:
#create and tune the models that you brainstormed during part 2

### ---- 9 Test models ----

In [1]:
#do 5-fold cross validation on models and measure MSE

### ---- 10 Select best model  ----

In [ ]:
#select the model with the lowest error as your "prodcuction" model

## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### ---- 12 Deploy solution ----

In [16]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data